In [1]:
from sklearn.preprocessing import StandardScaler

from influxdb.commands import InfluxDBQueries, DataProcessor
import warnings
import pandas as pd
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
df_train = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2024-05-01T01:00:00Z",
                                                stop="2025-01-01T01:00:00Z")
df_val = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-01-01T01:00:00Z",
                                              stop="2025-03-01T01:00:00Z")
df_test = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-03-01T01:00:00Z",
                                               stop="2025-04-01T01:00:00Z")

In [3]:
df_combined = pd.concat([df_train, df_test, df_val])

In [4]:
df_combined, scaler, columns, scaler_y = DataProcessor.add_features(df_combined)

In [6]:
df_combined.dropna(inplace=True)

In [ ]:
columns, boruta = DataProcessor.select_features_boruta(df_combined, columns)

In [ ]:
print(columns, boruta)

In [5]:
df_train = df_combined.loc[df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]
df_val = df_combined.loc[(df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')) &
                         (df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ'))]
df_test = df_combined.loc[df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]

In [6]:
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()

In [7]:
X_train, y_train= DataProcessor.make_sequences(df_train, columns_to_select=columns, window_size=288, forecast_horizon=1)

In [8]:
X_val, y_val= DataProcessor.make_sequences(df_val, columns_to_select=columns, window_size=288, forecast_horizon=1)

In [9]:
from influxdb.commands import LSTMModel

In [10]:
X_train.shape

(69778, 288, 47)

Based on the previous cells in the notebook, I can see that `X_train.shape` would reveal the correct shape for the input. Looking at how the data is processed, I need to match the shape with the columns used in the LSTM model.



In [11]:
lstm_forecaster = LSTMModel(input_shape=(288, len(columns)), output_steps=1)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 288, 128)       │        90,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 288, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 288, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 1, 1)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,401 (556.25 KB)

 Trainable params: 142,017 (554.75 KB)

 Non-trainable params: 384 (1.50 KB)

In [12]:
lstm_forecaster.fit(X_train, y_train, X_val,y_val, epochs=200, batch_size=128, patience=10)

Training started with EarlyStopping (patience=10)...
Epoch 1/200
546/546 ━━━━━━━━━━━━━━━━━━━━ 0s 879ms/step - directional_accuracy: 0.4997 - loss: 2.5424 - mse: 1.1536
Epoch 1: val_loss improved from inf to 1.98455, saving model to models/best_model.keras
546/546 ━━━━━━━━━━━━━━━━━━━━ 523s 954ms/step - directional_accuracy: 0.4997 - loss: 2.5420 - mse: 1.1533 - val_directional_accuracy: 0.4784 - val_loss: 1.9846 - val_mse: 0.9778 - learning_rate: 0.0010
Epoch 2/200
326/546 ━━━━━━━━━━━━━━━━━━━━ 3:11 872ms/step - directional_accuracy: 0.5010 - loss: 2.1351 - mse: 0.9214

KeyboardInterrupt: 

In [33]:
X_test, y_test = DataProcessor.make_sequences(df_test, columns_to_select=columns, window_size=288, forecast_horizon=1)

In [36]:
pred = lstm_forecaster.predict(X_test
                            )

270/270 ━━━━━━━━━━━━━━━━━━━━ 18s 65ms/step


In [28]:
negativ = 0
pozitiv = 0
for i in range(y_test.shape[0]):
    if y_test[i][0] < 0:
        negativ += 1
    else:
        pozitiv += 1

In [29]:
print(negativ)

4349


In [30]:
print(pozitiv)

4291


In [39]:
pred[30:60]

array([[[0.04604294]],

       [[0.04825704]],

       [[0.04473032]],

       [[0.03937104]],

       [[0.02771459]],

       [[0.03006771]],

       [[0.03491335]],

       [[0.0448788 ]],

       [[0.05020518]],

       [[0.05292452]],

       [[0.06357282]],

       [[0.07040992]],

       [[0.05456256]],

       [[0.06495886]],

       [[0.0608092 ]],

       [[0.05074956]],

       [[0.04879094]],

       [[0.04765058]],

       [[0.04650092]],

       [[0.04524057]],

       [[0.03762659]],

       [[0.02930048]],

       [[0.0260705 ]],

       [[0.01709907]],

       [[0.01418164]],

       [[0.00905424]],

       [[0.01213137]],

       [[0.01409099]],

       [[0.01730556]],

       [[0.02067445]]], dtype=float32)